# 假设

全眼的像差，是最终对视网膜产生影响的原因。那么使用佩戴OK镜之前、或者佩戴短期后的像差数据，是否可能预测出远期的眼轴长或者是屈光状态呢？

## 已知的缺陷

全眼像差受到多方面的影响：

* 瞳孔大小；
* 调节状态；
* 测量时间，OK镜佩戴后，白天的角膜形态是否会逐渐变化，导致像差随着时间改变。



# 数据

来自于[Predictive factors associated with axial length growth and myopia progression in orthokeratology ](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6561598/ )

该文献带有excel数据，共有7个sheet，分别是：

* age, sex, visual acuity：年龄，性别，视力，其中视力用LogMAR，包含了未矫正和最佳矫正视力。
* AXL：眼轴长，用IOL master测量了中央，鼻侧30度，颞侧30度
  >AXL measurement with IOLMaster (Carl Zeiss, Jena, Germany) in central, N30, and T30 gazes
* CR：散瞳验光，用WAM-5500测量了中央，鼻侧30度，颞侧30度
  >cycloplegic refraction; autorefraction (WAM-5500; Shigiya Machinery Works Ltd., Hiroshima, Japan) in central, 30° nasal (N30), and 30° temporal (T30) gazes under cycloplegia
* MR：
  >manifested refraction
* specular microscopy：不知为何，测量了角膜内皮细胞计数。
  >evaluation of the corneal endothelium via noncontact specular microscopy (SP-8000; Konan Medical, Nishinomiya, Japan). 
* aberrometer：像差，给了高阶的Zernike系数。
  >wavefront assessment for a 6-mm pupil using a WASCA aberrometer (Carl Zeiss, Jena, Germany) following pupil dilation using a mixture of 0.5% phenylephrine and 0.5% tropicamide (Mydrin-P; Santen Pharmaceutical, Osaka, Japan)
* pentacam：角膜地形图。
  很遗憾，这里面不是raw data，只有Pre和12mo的K1, K2
* orbscan II：角膜地形图。
  也不是角膜地形图的原始数据，但除了Kmin, Kmax,还有Central corneal thickness, 3-mm-zone irregularity, 5-mm-zone irregularity

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce


import os

%matplotlib inline

## 数据清洗

读取数据后，将数据分成两部分，（务必要注意是否有数据泄露）

* X：从这些数据可能推导出结果，我估计会有术前的数据，一部分术后的数据。
    * patient_info中：``` ['Age','Sex (male = 1, female = 2']```
    * AXL：```['Pre C AXL', 'Pre N AXL', 'Pre T AXL'] ```
    * CR：
      ```python
         ['Pre AR C Sph', 'Pre AR C cyl',\
          'Pre AR N Sph', 'Pre AR N cyl',\
          'Pre AR T Sph', 'Pre AR T cyl']  
       ```
    * aberrometer:
        * Pre和12mo （犹豫，不知道是否有数据泄露）
    * cornea：
        * Pre和12mo
* Y：
    * AXL：12mo的C，N，T，以及delta，其中delta 12mo C AXL是最重要的数据。

In [89]:
data_file=os.path.join('data',"pone.0218140.s001.xlsx")

patient_info=pd.read_excel(data_file,sheet_name="age, sex, visual acuity")
AXL=pd.read_excel(data_file,sheet_name="AXL")
CR=pd.read_excel(data_file,sheet_name="CR ")

# 以下两个sheet中，顶部有Pre，12mo一行，
# 略去，使得每一行与其他表格中的行位置相等。
aberrometer=pd.read_excel(data_file,sheet_name="aberrometer",header=1) 
cornea=pd.read_excel(data_file,sheet_name="orbscan II",header=1) 
data_frames=[patient_info,AXL,CR,aberrometer,cornea]

并不是所有的人都测量了所有的参数，所以将Patient ID和眼别整合到一起，形成一个新的eyeID。

In [91]:
for d in data_frames:
    d["Patient"].fillna(method='ffill',inplace = True)
    d["eyeID"]=d["Patient"]+" "+d['OD1, OS2'].map(str)


In [95]:
df = reduce(lambda left,right: pd.merge(left,right,on='eyeID'), data_frames)

In [96]:
df

,Patient_x,"Sex (male = 1, female = 2","OD1, OS2_x",Age,log UCVA,log BCVA,eyeID,Patient_y,"OD1, OS2_y",Pre C AXL,...,anterior chamber depth,Sim K's astigmatism.1,Kmax.1,Kmin.1,Central corneal thickness.1,3-mm-zone irregularity.1,5-mm-zone irregularity.1,pupil diameter.1,white-to-white.1,anterior chamber depth.1
0,#1,2.0,1.0,9.00000,0.698970,0.045757,#1 1.0,#1,1.0,23.570000,...,3.080000,0.700000,44.300000,43.600000,494.000000,1.900000,2.100000,5.000000,11.200000,2.980000
1,#1,NaN,2.0,NaN,0.301030,0.000000,#1 2.0,#1,2.0,23.460000,...,3.100000,1.300000,45.400000,44.100000,492.000000,1.800000,2.000000,5.000000,11.200000,2.960000
2,#2,2.0,1.0,9.00000,0.698970,0.000000,#2 1.0,#2,1.0,24.200000,...,3.090000,2.200000,42.300000,40.100000,549.000000,2.900000,3.500000,5.200000,11.600000,3.070000
3,#2,NaN,2.0,NaN,0.698970,0.045757,#2 2.0,#2,2.0,24.090000,...,3.040000,1.500000,42.500000,40.900000,553.000000,3.600000,3.400000,4.400000,11.500000,3.020000
4,#3,2.0,1.0,9.00000,0.522879,0.000000,#3 1.0,#3,1.0,24.230000,...,3.140000,0.700000,41.500000,40.800000,568.000000,2.600000,3.800000,4.400000,11.400000,3.110000
5,#3,NaN,2.0,NaN,0.522879,0.000000,#3 2.0,#3,2.0,24.110000,...,3.150000,0.400000,42.000000,41.600000,572.000000,1.400000,1.800000,4.600000,11.500000,3.100000
6,#4,2.0,1.0,9.00000,0.522879,0.000000,#4 1.0,#4,1.0,23.070000,...,3.130000,0.900000,44.200000,43.400000,511.000000,2.000000,2.300000,4.900000,11.100000,3.160000
7,#4,NaN,2.0,NaN,0.522879,0.000000,#4 2.0,#4,2.0,22.960000,...,2.990000,0.800000,44.000000,43.200000,519.000000,2.400000,2.900000,5.000000,11.200000,3.110000
8,#5,2.0,1.0,9.00000,0.522879,0.000000,#5 1.0,#5,1.0,24.900000,...,3.270000,0.600000,41.000000,40.400000,534.000000,3.300000,3.200000,4.900000,11.900000,3.260000
9,#5,NaN,2.0,NaN,0.522879,0.000000,#5 2.0,#5,2.0,24.860000,...,3.080000,1.500000,41.700000,40.200000,529.000000,6.200000,8.100000,5.200000,12.200000,3.250000
